In [4]:
import nltk 
import pandas as pd
from nltk.corpus import stopwords
#from textblob import Word
from sklearn.preprocessing import LabelEncoder
from collections import Counter
#import wordcloud


from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [210]:
#data = pd.read_csv('amazon_alexa.tsv', sep = '\t')
data = pd.read_csv('https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/amazon.csv')

In [211]:
data.head()

,reviewText,Positive
0,This is a one of the best apps acording to a b...,1
1,This is a pretty good version of the game for ...,1
2,this is a really cool game. there are a bunch ...,1
3,"This is a silly game and can be frustrating, b...",1
4,This is a terrific game on any pad. Hrs of fun...,1


In [212]:
#Check if there are any null values
data = data[data['reviewText'].notnull()]
data.isnull().sum()

reviewText    0
Positive      0
dtype: int64

In [149]:
data.shape

(20000, 4)

In [214]:
data.Positive.value_counts()/data.shape[0]

Positive
1    0.76165
0    0.23835
Name: count, dtype: float64

In [144]:
# def cleaning(df, stop_words):
#     from nltk.tokenize import RegexpTokenizer

#     #tokenizer = RegexpTokenizer(r'\w+')

#     df['verified_reviews'] = df['verified_reviews'].apply(lambda x: ' '.join(x.lower() for x in x.split()))
#     #df['verified_reviews'] = df['verified_reviews'].apply(lambda x: ' '.join(tokenizer.tokenize(x.lower())))
#     # Replacing the special characters

#     df['verified_reviews'] = df['verified_reviews'].str.replace('[^ws]', '')

#     # Replacing the digits/numbers

#     df['verified_reviews'] = df['verified_reviews'].str.replace('d', '')

#     # Removing stop words

#     df['verified_reviews'] = df['verified_reviews'].apply(lambda x:' '.join(x for x in x.split() if x not in stop_words))

#     # Lemmatization

#     df['verified_reviews'] = df['verified_reviews'].apply(lambda x:' '.join([Word(x).lemmatize() for x in x.split()]))
#     return df

# stop_words = stopwords.words('english')
# data_v1 = cleaning(data_v1, stop_words)

In [215]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

tokenizer = RegexpTokenizer(r'\w+')

df = data.copy()
df['reviewText'] = df['reviewText'].apply(lambda x: tokenizer.tokenize(x.lower()))
df['reviewText'] = df['reviewText'].apply(lambda x: [i for i in x if i not in stopwords.words('english')])
df['reviewText'] = df['reviewText'].apply(lambda x: [WordNetLemmatizer().lemmatize(i) for i in x])
df['sentiment'] = df['reviewText'].apply(lambda x: ' '.join(x))

In [216]:
df.head()

,reviewText,Positive,sentiment
0,"[one, best, apps, acording, bunch, people, agr...",1,one best apps acording bunch people agree bomb...
1,"[pretty, good, version, game, free, lot, diffe...",1,pretty good version game free lot different le...
2,"[really, cool, game, bunch, level, find, golde...",1,really cool game bunch level find golden egg s...
3,"[silly, game, frustrating, lot, fun, definitel...",1,silly game frustrating lot fun definitely reco...
4,"[terrific, game, pad, hr, fun, grandkids, love...",1,terrific game pad hr fun grandkids love great ...


In [170]:
# Initialize Sentiment Analyzer

from nltk.sentiment.vader import SentimentIntensityAnalyzer
'''
Sentiment analysis is a technique used to determine the emotional tone or sentiment expressed in a text. It involves analyzing 
the words and phrases used in the text to identify the underlying sentiment, whether it is positive, negative, or neutral.

Good for short text. 

'''
analyzer = SentimentIntensityAnalyzer()

def analyser(text):
    scores = analyzer.polarity_scores(text)
    sentiment = 1 if scores['pos'] > 0 else 0
    return sentiment

# Apply this to our dataframe

df['scores'] = df['sentiment'].apply(analyser)
print(df.head())

## Classification report

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print(confusion_matrix(df['Positive'], df['scores']))

from sklearn.metrics import classification_report
print(classification_report(df['Positive'], df['scores']))

                                          reviewText  Positive  \
0  [one, best, apps, acording, bunch, people, agr...         1   
1  [pretty, good, version, game, free, lot, diffe...         1   
2  [really, cool, game, bunch, level, find, golde...         1   
3  [silly, game, frustrating, lot, fun, definitel...         1   
4  [terrific, game, pad, hr, fun, grandkids, love...         1   

                                    verified_reviews  \
0  This is a one of the best apps acording to a b...   
1  This is a pretty good version of the game for ...   
2  this is a really cool game. there are a bunch ...   
3  This is a silly game and can be frustrating, b...   
4  This is a terrific game on any pad. Hrs of fun...   

                                           sentiment  scores  
0  one best apps acording bunch people agree bomb...       1  
1  pretty good version game free lot different le...       1  
2  really cool game bunch level find golden egg s...       1  
3  silly game 

In [217]:
df.head()

,reviewText,Positive,sentiment
0,"[one, best, apps, acording, bunch, people, agr...",1,one best apps acording bunch people agree bomb...
1,"[pretty, good, version, game, free, lot, diffe...",1,pretty good version game free lot different le...
2,"[really, cool, game, bunch, level, find, golde...",1,really cool game bunch level find golden egg s...
3,"[silly, game, frustrating, lot, fun, definitel...",1,silly game frustrating lot fun definitely reco...
4,"[terrific, game, pad, hr, fun, grandkids, love...",1,terrific game pad hr fun grandkids love great ...


### Countvectorizer

In [220]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

count_vectoriser = CountVectorizer()                                  ## countvectorizer model object
count_vectoriser2 = count_vectoriser.fit_transform(df['sentiment']) ## create sparse matrix of documents and vector counts

features = count_vectoriser.get_feature_names_out() #features

df_sparse = pd.DataFrame(count_vectoriser2.todense(), columns = features)

feature_counts = np.sum(df_sparse.values, axis = 0)

feature_counts_df = pd.DataFrame(dict(features = features, counts = feature_counts))
feature_counts_df.sort_values(by = 'counts', ascending=False, inplace = True); feature_counts_df.head()


,features,counts
1187,app,10879
6415,game,6899
9070,love,4302
6813,great,4092
8850,like,4042


### Naive-Bayes Modeling


In [221]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

countvect_tuned = CountVectorizer(stop_words='english', min_df=0.2, max_df=0.5, ngram_range=(1, 3)) 
#countvect_tuned = CountVectorizer(stop_words='english') 

#min_df - ignores tokens which are present in less than 20% documents.
#max_df - ignores tokens which are present in more than 50% of the document.
#max_features - limiting vocabulary size to 50. Selectly mots common 50 tokens.
#binary - if set to False, then counts aren't taken only 1 and 0 are imputed.
#processor = noise removal function etc.

countvect_tuned_vector = countvect_tuned.fit_transform(df['sentiment'])

features = countvect_tuned.get_feature_names_out() #features

df_sparse = pd.DataFrame(countvect_tuned_vector.todense(), columns = features)

feature_counts = np.sum(df_sparse.values, axis = 0)
feature_counts_df = pd.DataFrame(dict(features = features, counts = feature_counts))
feature_counts_df.sort_values(by = 'counts', ascending=False, inplace = True); feature_counts_df.head()

,features,counts
0,app,10879
1,game,6899


In [222]:
X_train, X_test, y_train, y_test = train_test_split(countvect_tuned_vector.toarray(), df['Positive'], test_size=0.2, random_state=1)
nb_clf = GaussianNB().fit(X_train, y_train)
y_pred = nb_clf.predict(X_test)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print('Train accuracy: {}'.format(accuracy_score(y_train, nb_clf.predict(X_train))))
print('Test accuracy: {}'.format(accuracy_score(y_test, y_pred)))
 
print(confusion_matrix(y_test, y_pred))

print(classification_report(y_test, y_pred))

Train accuracy: 0.76
Test accuracy: 0.763
[[   3  944]
 [   4 3049]]
              precision    recall  f1-score   support

           0       0.43      0.00      0.01       947
           1       0.76      1.00      0.87      3053

    accuracy                           0.76      4000
   macro avg       0.60      0.50      0.44      4000
weighted avg       0.68      0.76      0.66      4000



### TF_IDF

In [223]:
from sklearn.feature_extraction.text import TfidfVectorizer   

idf_vect = TfidfVectorizer(stop_words='english', min_df=0.01, max_df=.7, ngram_range=(1, 3))                            
idf_vect_2 = idf_vect.fit_transform(df['sentiment']) 

X_train, X_test, y_train, y_test = train_test_split(idf_vect_2.toarray(), df['Positive'], test_size=0.2, random_state=1)
nb_clf = GaussianNB().fit(X_train, y_train)
y_pred = nb_clf.predict(X_test)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print('Train accuracy: {}'.format(accuracy_score(y_train, nb_clf.predict(X_train))))
print('Test accuracy: {}'.format(accuracy_score(y_test, y_pred)))
 
print(confusion_matrix(y_test, y_pred))

print(classification_report(y_test, y_pred))

Train accuracy: 0.7678125
Test accuracy: 0.764
[[ 813  134]
 [ 810 2243]]
              precision    recall  f1-score   support

           0       0.50      0.86      0.63       947
           1       0.94      0.73      0.83      3053

    accuracy                           0.76      4000
   macro avg       0.72      0.80      0.73      4000
weighted avg       0.84      0.76      0.78      4000



In [227]:
df.head()

,reviewText,Positive,sentiment
0,"[one, best, apps, acording, bunch, people, agr...",1,one best apps acording bunch people agree bomb...
1,"[pretty, good, version, game, free, lot, diffe...",1,pretty good version game free lot different le...
2,"[really, cool, game, bunch, level, find, golde...",1,really cool game bunch level find golden egg s...
3,"[silly, game, frustrating, lot, fun, definitel...",1,silly game frustrating lot fun definitely reco...
4,"[terrific, game, pad, hr, fun, grandkids, love...",1,terrific game pad hr fun grandkids love great ...


In [229]:
# common_words=''
# for i in df.reviewText:
#     i = str(i)
#     tokens = i.split()
#     common_words += " ".join(tokens)+" "
# wordcloud = wordcloud.WordCloud().generate(common_words)
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.show()

In [201]:
# # Encoded the target column
# lb=LabelEncoder()
# df['sentiment'] = lb.fit_transform(df['Positive'])

In [5]:
from keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

ModuleNotFoundError: No module named 'keras'

In [232]:
tokenizer = Tokenizer(num_words=None, split=' ')               # It tokenizes by taking all unique values from the training text.
tokenizer.fit_on_texts(df['sentiment'].values)    # fit it on the text
X = tokenizer.texts_to_sequences(df['sentiment'].values)  # get corresponding sequnce number for the tokens
X = pad_sequences(X)                                                  # make the sequence to equal length with zeroes at the starting

#from keras.preprocessing.text import text_to_word_sequence - to split on spaces and lowrcase, remove punctuations,
#tokens = text_to_word_sequence(text)

word_index = tokenizer.word_index
print(len(word_index))

16833


In [236]:
X.shape

(20000, 42)

In [240]:
sequences = tokenizer.texts_to_sequences(["physics is nice jnhkjl love echo echo"]) # .
text = pad_sequences(sequences, maxlen=15, padding = 'pre')  # pre means filling zero at first and post means doing it at end.
print(text)

[[   0    0    0    0    0    0    0    0    0    0    0   63    3 4327
  4327]]


In [241]:
text.shape

(1, 15)

In [274]:
X

array([[    0,     0,     0, ...,   798,  8036,   246],
       [    0,     0,     0, ...,    88,    91,    30],
       [    0,     0,     0, ...,  1451,   316,    12],
       ...,
       [    0,     0,     0, ...,    25,    12,    12],
       [    0,     0,     0, ...,   508,   299,    13],
       [    0,     0,     0, ...,   182, 16833,   733]])

In [275]:
X[5]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,  400,    2,  545,   15,  408,
          5,   17,   12,    2, 1659, 2248,    2,  126,  261])

In [276]:
X.shape[1]

42

In [279]:

#Splitting the data into training and testing
y=pd.get_dummies(df['Positive'])
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)
from keras.layers import LSTM, GRU,SimpleRNN


In [282]:
X_train

array([[   0,    0,    0, ...,   30,   36,  246],
       [   0,    0,    0, ..., 5338,  105,    5],
       [   0,    0,    0, ...,   38,  171,  109],
       ...,
       [   0,    0,    0, ...,   43,    9, 5520],
       [   0,    0,    0, ...,   19,  859,  711],
       [   0,    0,    0, ...,  232, 2156, 2206]])

### SimpleRNN - No weights

In [283]:
model = Sequential()
model.add(Embedding(len(word_index)+1, 300, input_shape = (X.shape[1],)))

#Embedding layer learns weights along it's way.And converts discrete data to real valued densed vector.

model.add(SimpleRNN(305, dropout=0.2, recurrent_dropout=0.2))

model.add(Dense(2,activation='softmax'))
# predict if 1 or 0. Binary 

model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
# compile the model 

print(model.summary())

# Fitting on train
batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

print('Simple RNN LSTM Performance')
model.evaluate(X_test,y_test)

C:\Users\p192\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_24 (Embedding)             │ (None, 42, 300)             │       5,050,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_4 (SimpleRNN)             │ (None, 305)                 │         184,830 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 2)                   │             612 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,235,642 (19.97 MB)

 Trainable params: 5,235,642 (19.97 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 78s 145ms/step - accuracy: 0.7289 - loss: 0.5848
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 47s 93ms/step - accuracy: 0.8191 - loss: 0.4032
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 46s 92ms/step - accuracy: 0.9126 - loss: 0.2226
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 45s 89ms/step - accuracy: 0.9378 - loss: 0.1630
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 44s 88ms/step - accuracy: 0.9539 - loss: 0.1262
Simple RNN LSTM Performance
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.8558 - loss: 0.3903


[0.3898458480834961, 0.8544999957084656]

### LSTM - No weights

In [284]:
model = Sequential()
model.add(Embedding(len(word_index)+1, 300, input_shape = (X.shape[1],)))
# takes in input length 500, it can be no longer than the highest tokenized index that is 500 or max unique values. 2d tensor (batch, input_dim)
# outputs output of length 120, 3D tensor (batch, input_shape or max_text_length, output_dim). Batch = None

# It will output texts with max_text_length size each embedded to 120 dimensions.

model.add(SpatialDropout1D(0.3))
# inputs 3D tensor, drops 40% input units.

model.add(LSTM(305, dropout=0.2, recurrent_dropout=0.2))
# 3D tensor now fed to LSTM with 176 units. Returns (Batch, 176 outputs)
# 20% Fraction of the units to drop for the linear transformation of the inputs.
# 20% Fraction of the units to drop for the linear transformation of the recurrent state.

model.add(Dense(2,activation='softmax'))
# predict if 1 or 0. Binary 

model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
# compile the model 

print(model.summary())

# Fitting on train
batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

print('LSTM Performance')
model.evaluate(X_test,y_test)

C:\Users\p192\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_25 (Embedding)             │ (None, 42, 300)             │       5,050,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d_6                  │ (None, 42, 300)             │               0 │
│ (SpatialDropout1D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ (None, 305)                 │         739,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 2)                   │             612 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,790,132 (22.09 MB)

 Trainable params: 5,790,132 (22.09 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 92s 180ms/step - accuracy: 0.8235 - loss: 0.3997
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 97s 194ms/step - accuracy: 0.9350 - loss: 0.1747
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 95s 190ms/step - accuracy: 0.9575 - loss: 0.1188
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 102s 205ms/step - accuracy: 0.9698 - loss: 0.0813
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 101s 203ms/step - accuracy: 0.9822 - loss: 0.0539
LSTM Performance
125/125 ━━━━━━━━━━━━━━━━━━━━ 9s 68ms/step - accuracy: 0.8808 - loss: 0.4613


[0.48737993836402893, 0.878000020980835]

### GRU - NO WEIGHTS

In [285]:
from keras.layers import LSTM, GRU,SimpleRNN

model = Sequential()
model.add(Embedding(len(word_index)+1, 300, input_shape = (X.shape[1],)))
# takes in input dim = max uniqe values, it can be no longer than the highest tokenized index or max unique values. 2d tensor (batch, input_dim)
# outputs output of length 120, 3D tensor (batch, input_shape or max_text_length, output_dim). Batch = None

# It will output texts with max_text_length size each embedded to 120 dimensions.

model.add(SpatialDropout1D(0.3))
# inputs 3D tensor, drops 40% input units.

model.add(GRU(305, dropout=0.2, recurrent_dropout=0.2))

model.add(Dense(2,activation='softmax'))
# predict if 1 or 0. Binary 

model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
# compile the model 

print(model.summary())

# Fitting on train
batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

print('GRU Performance')
model.evaluate(X_test,y_test)

C:\Users\p192\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_26 (Embedding)             │ (None, 42, 300)             │       5,050,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d_7                  │ (None, 42, 300)             │               0 │
│ (SpatialDropout1D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 305)                 │         555,405 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 2)                   │             612 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,606,217 (21.39 MB)

 Trainable params: 5,606,217 (21.39 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 81s 156ms/step - accuracy: 0.8180 - loss: 0.4198
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 79s 158ms/step - accuracy: 0.9333 - loss: 0.1793
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 80s 160ms/step - accuracy: 0.9574 - loss: 0.1195
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 81s 162ms/step - accuracy: 0.9728 - loss: 0.0867
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 83s 167ms/step - accuracy: 0.9801 - loss: 0.0583
GRU Performance
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - accuracy: 0.8488 - loss: 0.4222


[0.4430953562259674, 0.8402500152587891]

### BRNN - NO WEIGHTS

In [286]:
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D 

model = Sequential()
model.add(Embedding(len(word_index)+1, 300, input_shape = (X.shape[1],)))

model.add(Bidirectional(LSTM(305, dropout=0.2, recurrent_dropout=0.2)))

model.add(Dense(2,activation='softmax'))
# predict if 1 or 0. Binary 

model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
# compile the model 

print(model.summary())

# Fitting on train
batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

print('BRNN LSTM Performance')
model.evaluate(X_test,y_test)

C:\Users\p192\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_27"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_27 (Embedding)             │ (None, 42, 300)             │       5,050,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 610)                 │       1,478,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 2)                   │           1,222 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,530,062 (24.91 MB)

 Trainable params: 6,530,062 (24.91 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 136s 266ms/step - accuracy: 0.8344 - loss: 0.3849
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 161s 322ms/step - accuracy: 0.9391 - loss: 0.1629
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 155s 310ms/step - accuracy: 0.9668 - loss: 0.1008
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 161s 323ms/step - accuracy: 0.9764 - loss: 0.0685
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 161s 323ms/step - accuracy: 0.9817 - loss: 0.0540
BRNN LSTM Performance
125/125 ━━━━━━━━━━━━━━━━━━━━ 19s 147ms/step - accuracy: 0.8785 - loss: 0.4289


[0.4484785199165344, 0.8759999871253967]

### Using Glove pretrained embedddings

In [263]:
num_tokens = len(word_index) + 2
embedding_dim = 300
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 14654 words (2179 misses)


In [269]:
model = Sequential()
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index)+2, 300, input_shape = (X.shape[1],))
embedding_layer.build((1,))
embedding_layer.set_weights([embedding_matrix])

model.add(embedding_layer)

model.add(SpatialDropout1D(0.3))
# inputs 3D tensor, drops 40% input units.

model.add(LSTM(300, dropout=0.2, recurrent_dropout=0.2))
# 3D tensor now fed to LSTM with 176 units. Returns (Batch, 176 outputs)
# 20% Fraction of the units to drop for the linear transformation of the inputs.
# 20% Fraction of the units to drop for the linear transformation of the recurrent state.

model.add(Dense(2,activation='softmax'))
# predict if 1 or 0. Binary 

model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
# compile the model 



print(model.summary())

#Splitting the data into training and testing
y=pd.get_dummies(data_v1['sentiment'])
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)

# Fitting on train
batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

print('LSTM Performance')
model.evaluate(X_test,y_test)

Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_22 (Embedding)             │ (None, 42, 300)             │       5,050,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d_5                  │ (None, 42, 300)             │               0 │
│ (SpatialDropout1D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 300)                 │         721,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 2)                   │             602 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,772,302 (22.02 MB)

 Trainable params: 5,772,302 (22.02 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 261s 574ms/step - accuracy: 0.8254 - loss: 0.3893
Epoch 2/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 246s 561ms/step - accuracy: 0.8998 - loss: 0.2339
Epoch 3/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 244s 557ms/step - accuracy: 0.9326 - loss: 0.1661
Epoch 4/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 243s 555ms/step - accuracy: 0.9563 - loss: 0.1168
Epoch 5/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 240s 547ms/step - accuracy: 0.9667 - loss: 0.0915
LSTM Performance
188/188 ━━━━━━━━━━━━━━━━━━━━ 42s 218ms/step - accuracy: 0.8996 - loss: 0.3124


[0.31509363651275635, 0.8958333134651184]

### BRNN - GOLVE WEIGHTS

In [289]:
model = Sequential()
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index)+2, 300, input_shape = (X.shape[1],))
embedding_layer.build((1,))
embedding_layer.set_weights([embedding_matrix])

model.add(embedding_layer)

model.add(SpatialDropout1D(0.3))
# inputs 3D tensor, drops 40% input units.

model.add(Bidirectional(LSTM(305, dropout=0.2, recurrent_dropout=0.2)))
# 3D tensor now fed to LSTM with 176 units. Returns (Batch, 176 outputs)
# 20% Fraction of the units to drop for the linear transformation of the inputs.
# 20% Fraction of the units to drop for the linear transformation of the recurrent state.

model.add(Dense(2,activation='softmax'))
# predict if 1 or 0. Binary 

model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
# compile the model 



print(model.summary())

#Splitting the data into training and testing
y=pd.get_dummies(data_v1['sentiment'])
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)

# Fitting on train
batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

print('BRNN GLOVE Performance')
model.evaluate(X_test,y_test)

C:\Users\p192\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_28"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_28 (Embedding)             │ (None, 42, 300)             │       5,050,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d_8                  │ (None, 42, 300)             │               0 │
│ (SpatialDropout1D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 610)                 │       1,478,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 2)                   │           1,222 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,530,362 (24.91 MB)

 Trainable params: 6,530,362 (24.91 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 124s 274ms/step - accuracy: 0.8260 - loss: 0.3896
Epoch 2/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 122s 279ms/step - accuracy: 0.9039 - loss: 0.2337
Epoch 3/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 123s 281ms/step - accuracy: 0.9295 - loss: 0.1744
Epoch 4/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 132s 302ms/step - accuracy: 0.9521 - loss: 0.1201
Epoch 5/5
438/438 ━━━━━━━━━━━━━━━━━━━━ 226s 518ms/step - accuracy: 0.9680 - loss: 0.0864
BRNN GLOVE Performance
188/188 ━━━━━━━━━━━━━━━━━━━━ 51s 264ms/step - accuracy: 0.9009 - loss: 0.3019


[0.31185993552207947, 0.8970000147819519]

In [287]:
len(word_index)+1

16834

In [288]:
X.shape

(20000, 42)

### Attention

In [50]:
from keras.layers import Layer
import keras.backend as K
from tensorflow import keras

In [96]:
from keras.layers import Layer
import keras.backend as K
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")     
        
        #will define weights and biases of size(out_dimensions from lstm, 1)
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(np.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)
        # here we do inner dot product of input with weights and biases and pass over tan and softmax which give alignment scores
        #whose dimensions will be max_len = input_shape, 42 which will be dot product with input to get context vector.

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
model = Sequential()
model.add(Embedding(len(word_index)+1, 300, input_shape = (X.shape[1],)))
model.add(SpatialDropout1D(0.3))
# inputs 3D tensor, drops 40% input units.

model.add(LSTM(305, dropout=0.2, recurrent_dropout=0.2, return_sequences = True))
model.add(attention())
# 3D tensor now fed to LSTM with 176 units. Returns (Batch, 176 outputs)
# 20% Fraction of the units to drop for the linear transformation of the inputs.
# 20% Fraction of the units to drop for the linear transformation of the recurrent state.

model.add(Dense(2,activation='softmax'))
# predict if 1 or 0. Binary 

model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
# compile the model 

print(model.summary())

# Fitting on train
batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

print('LSTM Performance')
model.evaluate(X_test,y_test)

In [290]:
(X.shape[1],)

(42,)

### BERT

Step 4: Preprocessing cleaning

Step 4: Tokenization & Encoding
BERT tokenization is used to convert the raw text into numerical inputs that can be fed into the BERT model. It tokenized the text and performs some preprocessing to prepare the text for the model’s input format. Let’s understand some of the key features of the BERT tokenization model.

BERT tokenizer splits the words into subwords or workpieces. For example, the word “geeksforgeeks” can be split into “geeks” “##for”, and”##geeks”. The “##” prefix indicates that the subword is a continuation of the previous one. It reduces the vocabulary size and helps the model to deal with rare or unknown words.

BERT tokenizer adds special tokens like [CLS], [SEP], and [MASK] to the sequence. These tokens have special meanings like :

[CLS] is used for classifications and to represent the entire input in the case of sentiment analysis,
[SEP] is used as a separator i.e. to mark the boundaries between different sentences or segments,
[MASK] is used for masking i.e. to hide some tokens from the model during pre-training.

BERT tokenizer gives their components as outputs:

input_ids: numerical tags to vocabulary tokens
token_type_ids: It identifies which segment or sentence each token belongs to.
attention_mask: It flags that inform the model which tokens to pay attention to and which to disregard.

When training language models, defining a prediction goal is a challenge. Many models predict the next word in a sequence, which is a directional approach and may limit context learning. BERT addresses this challenge with two innovative training strategies:

Masked Language Model (MLM)
Next Sentence Prediction (NSP)


1) MLM

the real challenge is figuring out the missing parts, and this strategy helps BERT become really good at understanding the meaning and context of words.

It masks around 15% tokens and model aims at guessing this word. 

The output vectors from the classification layer are multiplied by the embedding matrix, transforming them into the vocabulary dimension. This step helps align the predicted representations with the vocabulary space.

encoder output is passed over softmax to make probabilities over entire vocabulary for masked position.

The model converges slower than directional models. This is because, during training, BERT is only concerned with predicting the masked values, ignoring the prediction of the non-masked words. The increased context awareness achieved through this strategy compensates for the slower convergence.

2) NSP
model predicts if second sentence is follweed by first.

<cls> sent1 <sep> sent2
model's output predicts cls probabilty in 2*1 shaped vector. 
    
During training, 50% correct pairs and 50% random pairs are fed and trained.
    
A sentence embedding indicating Sentence A or Sentence B is added to each token. A positional encoding is also added.
    
    
NLM and NSP are trained simultaneously.
    
NER - each token gets convered to vectors and tags are asscoiated.
    
question answering - BERT is trained for question answering by learning two additional vectors that mark the beginning and end of the answer. During training, the model is provided with questions and corresponding passages, and it learns to predict the start and end positions of the answer within the passage.
    
    


In [ ]:
## Imports libs
from transformers import BertTokenizer, TFBertForSequenceClassification
import pandas as pd
import re


import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
from wordcloud import WordCloud, STOPWORDS
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
#Tokenize and encode the data using the BERT tokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) #uncased version lowers words before feeding to wordtokenizer


max_len= 128
# Tokenize and encode the sentences
X_train_encoded = tokenizer.batch_encode_plus(Reviews.tolist(),
                                              padding=True, 
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')
 
X_val_encoded = tokenizer.batch_encode_plus(x_val.tolist(), 
                                              padding=True, 
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')
 
X_test_encoded = tokenizer.batch_encode_plus(x_test.tolist(), 
                                              padding=True, 
                                              truncation=True,
                                              max_length = max_len,
                                              return_tensors='tf')


k = 0
print('Training Comments -->>',Reviews[k])

'''Valentine is a horrible movie This is what I thought of itActing Very bad Katherine Heigl can not act 
The other's weren't much betterStory The story was okay, but it could have been more developed This movie had the potential to be a great movie, 
but it failedMusic Yes, some of the music was pretty coolOriginality Not very original The name Paige Prescott' Recognize PrescottBottom Line 
Don't see Valentine It's a really stupid movie110'''

print('\nInput Ids -->>\n',X_train_encoded['input_ids'][k])
'''
 tf.Tensor(
[  101 10113  2003  1037  9202  3185  2023  2003  2054  1045  2245  1997
  2009 18908  2075  2200  2919  9477  2002  8004  2140  2064  2025  2552
  1996  2060  1005  1055  4694  1005  1056  2172  2488 23809  2100  1996
  2466  2001  3100  1010  2021  2009  2071  2031  2042  2062  2764  2023
  3185  2018  1996  4022  2000  2022  1037  2307  3185  1010  2021  2009
  3478 27275  2748  1010  2070  1997  1996  2189  2001  3492  4658 10050
 24965  3012  2025  2200  2434  1996  2171 17031 20719  1005  6807 20719
 18384 20389  2240  2123  1005  1056  2156 10113  2009  1005  1055  1037
  2428  5236  3185 14526  2692   102     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0], shape=(128,), dtype=int32)
'''

print('\nDecoded Ids -->>\n',tokenizer.decode(X_train_encoded['input_ids'][k]))
'''
[CLS] valentine is a horrible movie this is what i thought of itacting very bad katherine heigl can not act 
the other's weren't much betterstory the story was okay, but it could have been more developed this movie had the potential to be a great movie,
 but it failedmusic yes, some of the music was pretty cooloriginality not very original the name paige prescott'recognize prescottbottom line 
don't see valentine it's a really stupid movie110 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
'''

print('\nAttention Mask -->>\n',X_train_encoded['attention_mask'][k])
'''
 tf.Tensor(
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(128,), dtype=int32)
'''

print('\nLabels -->>',Target[k])
'''
Labels -->> 0
'''

# Intialize the model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Compile the model with an appropriate optimizer, loss function, and metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])


# Step 5: Train the model
history = model.fit(
    [X_train_encoded['input_ids'], X_train_encoded['token_type_ids'], X_train_encoded['attention_mask']],
    Target,
    validation_data=(
      [X_val_encoded['input_ids'], X_val_encoded['token_type_ids'], X_val_encoded['attention_mask']],y_val),
    batch_size=32,
    epochs=3
)


Epoch 1/3
782/782 [==============================] - 513s 587ms/step - loss: 0.3445 - accuracy: 0.8446 - val_loss: 0.2710 - val_accuracy: 0.8880
Epoch 2/3
782/782 [==============================] - 432s 552ms/step - loss: 0.2062 - accuracy: 0.9186 - val_loss: 0.2686 - val_accuracy: 0.8886
Epoch 3/3
782/782 [==============================] - 431s 551ms/step - loss: 0.1105 - accuracy: 0.9615 - val_loss: 0.3235 - val_accuracy: 0.8908


#Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(
    [X_test_encoded['input_ids'], X_test_encoded['token_type_ids'], X_test_encoded['attention_mask']],
    y_test
)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

391/391 [==============================] - 67s 171ms/step - loss: 0.3417 - accuracy: 0.8873
Test loss: 0.3417432904243469, Test accuracy: 0.8872799873352051
        
        
# Save the mdel and tokenizer to local
path = 'path-to-save'

# Save tokenizer
tokenizer.save_pretrained(path +'/Tokenizer')
# Save model
model.save_pretrained(path +'/Model')

# Load the model and tokenizer

# Load tokenizer
bert_tokenizer = BertTokenizer.from_pretrained(path +'/Tokenizer')
# Load model
bert_model = TFBertForSequenceClassification.from_pretrained(path +'/Model')

# Predict the sentiment of test dataset

pred = bert_model.predict(
    [X_test_encoded['input_ids'], X_test_encoded['token_type_ids'], X_test_encoded['attention_mask']])
 
# pred is of type TFSequenceClassifierOutput
logits = pred.logits
 
# Use argmax along the appropriate axis to get the predicted labels
pred_labels = tf.argmax(logits, axis=1)
 
# Convert the predicted labels to a NumPy array
pred_labels = pred_labels.numpy()
 
label = {
    1: 'positive',
    0: 'Negative'
}
 
# Map the predicted labels to their corresponding strings using the label dictionary
pred_labels = [label[i] for i in pred_labels]
Actual = [label[i] for i in y_test]
 
print('Predicted Label :', pred_labels[:10])
print('Actual Label    :', Actual[:10])

391/391 [==============================] - 68s 167ms/step
Predicted Label : ['positive', 'positive', 'positive', 'positive', 'positive', 'Negative', 'Negative', 'Negative', 'positive', 'positive']
Actual Label    : ['positive', 'positive', 'positive', 'Negative', 'positive', 'Negative', 'Negative', 'Negative', 'positive', 'positive']
    
print("Classification Report: \n", classification_report(Actual, pred_labels))

Classification Report: 
               precision    recall  f1-score   support
    Negative       0.91      0.86      0.88      6250
    positive       0.87      0.91      0.89      6250
    accuracy                           0.89     12500
    macro avg      0.89      0.89      0.89     12500
weighted avg       0.89      0.89      0.89     12500



Step 7: Prediction with user inputs



def Get_sentiment(Review, Tokenizer=bert_tokenizer, Model=bert_model):
    # Convert Review to a list if it's not already a list
    if not isinstance(Review, list):
        Review = [Review]
 
    Input_ids, Token_type_ids, Attention_mask = Tokenizer.batch_encode_plus(Review,
                                                                             padding=True,
                                                                             truncation=True,
                                                                             max_length=128,
                                                                             return_tensors='tf').values()
    prediction = Model.predict([Input_ids, Token_type_ids, Attention_mask])
 
    # Use argmax along the appropriate axis to get the predicted labels
    pred_labels = tf.argmax(prediction.logits, axis=1)
 
    # Convert the TensorFlow tensor to a NumPy array and then to a list to get the predicted sentiment labels
    pred_labels = [label[i] for i in pred_labels.numpy().tolist()]
    return pred_labels



Review ='''Bahubali is a blockbuster Indian movie that was released in 2015. 
It is the first part of a two-part epic saga that tells the story of a legendary hero who fights for his kingdom and his love. 
The movie has received rave reviews from critics and audiences alike for its stunning visuals, 
spectacular action scenes, and captivating storyline., 'negative'''


Get_sentiment(Review)

1/1 [==============================] - 3s 3s/step
['positive']

Accuracy of pre-trained bert-base-uncased Model on raw text data: 92.00%
Accuracy of pre-trained bert-base-uncased Model on pre-processed data: 89.00%

When I trained the above model on pre-processed data, I was getting an accuracy of 89%, but on the raw text data and with early stopping, I got an accuracy of 92%.